In [3]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

Deploy Panel apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


# Clase ternaria para mayo/junio

In [4]:
base_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/'
dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_crudo.csv'


ganancia_acierto = 273000
costo_estimulo = 7000

In [5]:
%%sql
create or replace table competencia_02_crudo as
select
    numero_de_cliente, foto_mes
from read_csv_auto("{{dataset_path + dataset_file}}")

,Success


In [6]:
%%sql
select * from competencia_02_crudo
where foto_mes = 202106

,numero_de_cliente,foto_mes
0,249221109,202106
1,249221468,202106
2,249223005,202106
3,249228180,202106
4,249232117,202106
...,...,...
164871,1598444941,202106
164872,1598471047,202106
164873,1598515897,202106
164874,1598517059,202106


In [7]:
%%sql
create or replace table competencia_02_expColab as
with periodos as (
  select distinct foto_mes from competencia_02_crudo
), clientes as (
  select distinct numero_de_cliente from competencia_02_crudo
), todo as (
  select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
  select
  c.*
    , if(c.numero_de_cliente is null, 0, 1) as mes_0
  , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
  , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
  , case 
      when mes_2 = 1 then 'CONTINUA'
      when mes_1 = 1 and mes_2 = 0 then 'BAJA+2'
      when mes_1 = 0 and mes_2 = 0 then 'BAJA+1'
      when mes_1 = 0 and mes_2 = 1 then 'CONTINUA'
      else null
    end as clase_ternaria
  from todo t
  left join competencia_02_crudo c using (numero_de_cliente, foto_mes)
) 
select
* EXCLUDE (mes_0, mes_1, mes_2)
from clase_ternaria
where mes_0 = 1

,Success


In [9]:
%%sql
create or replace table competencia_02_expColab as
select numero_de_cliente
,foto_mes
,clase_ternaria 
from competencia_02_expColab


,Success


In [10]:
%%sql
PIVOT competencia_02_expColab
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes


,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,201901,688,718,123346
1,201902,720,688,124391
2,201903,688,760,124988
3,201904,759,579,125658
4,201905,580,660,126419
5,201906,662,608,127916
6,201907,609,689,129426
7,201908,683,552,131429
8,201909,553,576,133185
9,201910,583,624,135475


In [11]:
%%sql
COPY competencia_02_expColab TO '../../../datasets/competencia_01_expColab.csv' (FORMAT CSV, HEADER TRUE)

,Success


In [12]:
dataset_file = "competencia_01.csv"
dataset_file_2 ="competencia_01_expColab.csv"## Bajas junio

competencia_1=pd.read_csv(dataset_path + dataset_file)
bajas_competencia_1=pd.read_csv(dataset_path + dataset_file_2)

In [24]:
competencia_1.groupby("foto_mes")["clase_ternaria"].value_counts().reset_index()

,foto_mes,clase_ternaria,count
0,202101,CONTINUA,160606
1,202101,BAJA+2,785
2,202101,BAJA+1,635
3,202102,CONTINUA,160844
4,202102,BAJA+2,1017
5,202102,BAJA+1,785
6,202103,CONTINUA,161684
7,202103,BAJA+1,1020
8,202103,BAJA+2,981
9,202104,CONTINUA,161919


In [25]:
bajas_competencia_1.groupby("foto_mes")["clase_ternaria"].value_counts().reset_index().tail(20)

,foto_mes,clase_ternaria,count
70,202012,BAJA+1,649
71,202012,BAJA+2,634
72,202101,CONTINUA,160606
73,202101,BAJA+2,785
74,202101,BAJA+1,635
75,202102,CONTINUA,160844
76,202102,BAJA+2,1017
77,202102,BAJA+1,785
78,202103,CONTINUA,161684
79,202103,BAJA+1,1020


In [13]:
bajas_junio=bajas_competencia_1[bajas_competencia_1["foto_mes"]== 202106][["numero_de_cliente","clase_ternaria"]]

In [14]:
import os 
base_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/'
dataset_path = os.path.join(base_path, 'datasets/')

file_name = 'Competencia_01_tagg.csv'
file_path = os.path.join(dataset_path, file_name)


bajas_junio.to_csv(file_path, index=False)